# Análise de Queimadas no Brasil (2020)

## Resumo

Neste notebook, iremos explorar dados territoriais e de queimadas no Brasil utilizando a linguagem Python e as bibliotecas Folium, Shapely e GeoPandas.

**Anãlise de Dados Geográficos**

A Biblioteca beautifulsoup é utilizado para fazer webscraping

In [ ]:
!pip install beautifulsoup4

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import contextily
import folium
import matplotlib.pyplot as plt
import seaborn as sns

import os
from urllib.request import urlretrieve
import requests
import zipfile
from bs4 import BeautifulSoup

Formatando os formatos dos Gráficos

In [ ]:
sns.set(font_scale=1.4)
sns.set_style({'font.family': 'serif',
               'fontname': 'Times New Roman'})

## 1) Fontes de dados

### 1.1) Queimadas no Brasil em 2020

* Fonte: [Instituto Nacional de Pesquisas Espaciais (INPE)](http://queimadas.dgi.inpe.br/queimadas)

In [ ]:
df = pd.read_csv('../input/queimadas-brasil-2020/Focos_2020-01-01_2020-12-31.csv')

In [ ]:
type(df)

Visualizando os primeiros dados

In [ ]:
df.head()

shape dos dados(dimensões)

In [ ]:
df.shape

Definindo o Geodataframe com shapely

In [ ]:
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

Sistema de coordenada -> crs EPSG:4326 -> sistema de coordenadas em graus

In [ ]:
queimadas_brasil_2020 = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry=geometry)

In [ ]:
queimadas_brasil_2020.head()

In [ ]:
queimadas_brasil_2020.shape

Mudando o Sistema de Coordenada

In [ ]:
queimadas_brasil_2020 = queimadas_brasil_2020.to_crs(crs='EPSG:3857')

In [ ]:
queimadas_brasil_2020.crs.to_string()

In [ ]:
queimadas_brasil_2020.head()

Plotando mapa com matplolib

In [ ]:
ax = queimadas_brasil_2020.plot(figsize=(15, 10), markersize=1, color='red')
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

### 1.2) Malhas municipais do Brasil

* Fonte: [Instituto Brasileiro de Geografia e Estatística (IBGE)](https://www.ibge.gov.br/geociencias/downloads-geociencias.html)

Criando uma pasta

In [ ]:
if not os.path.exists('/kaggle/working/mapas'):
    os.makedirs('/kaggle/working/mapas')

Webscraping do site do IBGE

In [ ]:
url_brasil_malhas_municipios = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/Brasil/BR/BR_Municipios_2020.zip'

In [ ]:
%%time

urlretrieve(url_brasil_malhas_municipios, '/kaggle/working/mapas/BR_Municipios_2020.zip')

In [ ]:
with zipfile.ZipFile('/kaggle/working/mapas/BR_Municipios_2020.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/mapas/')

In [ ]:
os.remove('/kaggle/working/mapas/BR_Municipios_2020.zip')

Lendo dados do tipo shapeli

In [ ]:
brasil_malhas_municipios = gpd.read_file('/kaggle/working/mapas/BR_Municipios_2020.shp')

In [ ]:
brasil_malhas_municipios.shape

In [ ]:
brasil_malhas_municipios.head()

In [ ]:
brasil_malhas_municipios.iloc[0].geometry

In [ ]:
brasil_malhas_municipios.crs.to_string()

Transformando o sistema de coordenadas de graus para metros

In [ ]:
brasil_malhas_municipios.to_crs(epsg=3857, inplace=True)

In [ ]:
brasil_malhas_municipios.crs.to_string()

In [ ]:
ax = brasil_malhas_municipios.plot(figsize=(15, 10), edgecolor='black', facecolor='none')
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

### 1.3) Localidades brasileiras

* Fonte: [Instituto Brasileiro de Geografia e Estatística (IBGE)](https://www.ibge.gov.br/geociencias/downloads-geociencias.html)

In [ ]:
url_brasil_localidades = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades/Shapefile_SHP/'

In [ ]:
req = requests.get(url_brasil_localidades)

In [ ]:
dados_ibge = BeautifulSoup(req.text, "html.parser")
dados_ibge

Baixando os datasets

In [ ]:
%%time

for arq in dados_ibge.find_all('a'):
    if 'BR_Localidades' in arq.get_text():
        print(arq.get_text())
        req = requests.get(url_brasil_localidades + arq["href"])
        print(req.status_code)
        open('/kaggle/working/mapas/' + arq['href'], 'wb').write(req.content)

In [ ]:
brasil_localidades = gpd.read_file('/kaggle/working/mapas/BR_Localidades_2010_v1.shp')

In [ ]:
brasil_localidades.shape

In [ ]:
brasil_localidades.head()

O EPGSG -> é o sistema de coordenada de graus para metros

In [ ]:
brasil_localidades.crs.to_string()

In [ ]:
brasil_localidades.to_crs(epsg=3857, inplace=True)

In [ ]:
ax = brasil_localidades.plot(figsize=(15, 10), markersize=1, color='blue')
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

## 2) O que podemos extrair de informações dos dados?

### 2.1) Queimadas e biomas

In [ ]:
queimadas_brasil_2020.head()

Bioma Unicos

In [ ]:
queimadas_brasil_2020['bioma'].unique()

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(x='bioma', data=queimadas_brasil_2020)
ax.set_xlabel('Bioma')
ax.set_ylabel('Qtd. de queimadas')
plt.show()

In [ ]:
ax = queimadas_brasil_2020.plot(figsize=(15, 10), column='bioma', markersize=1, legend=True)
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

### 2.2) Filtrando e visualizando o risco de fogo

In [ ]:
queimadas_brasil_2020.head()

Verificando o risco de fogo

In [ ]:
queimadas_brasil_2020['riscofogo'] >= 1

In [ ]:
queimadas_risco_alto = queimadas_brasil_2020[queimadas_brasil_2020['riscofogo'] >= 1]
queimadas_risco_alto.head()

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(x='bioma', data=queimadas_risco_alto)
ax.set_xlabel('Bioma')
ax.set_ylabel('Qtd. de queimadas')
plt.show()

Plotando as informações

In [ ]:
ax = queimadas_risco_alto.plot(figsize=(15, 10), column='bioma', markersize=1, legend=True)
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

### 2.3) Entendendo as categorias das localidades

In [ ]:
brasil_localidades.head()

Verificando as colunas do dataset

In [ ]:
brasil_localidades.columns

In [ ]:
brasil_localidades['NM_CATEGOR'].unique()

In [ ]:
plt.figure(figsize=(20,5))
ax = sns.countplot(x='NM_CATEGOR', data=brasil_localidades)
ax.set_xlabel('Categoria')

labels = [l.replace(' ', '\n') for l in brasil_localidades['NM_CATEGOR'].unique()]
ax.set_xticklabels(labels=labels)

ax.set_ylabel('Quantidade')
plt.show()

In [ ]:
aldeias_indigenas = brasil_localidades[brasil_localidades['NM_CATEGOR'] == 'ALDEIA INDÍGENA']
aldeias_indigenas.head()

In [ ]:
ax = aldeias_indigenas.plot(figsize=(15, 10), markersize=1, color='orange')
contextily.add_basemap(ax)
ax.set_axis_off()
plt.show()

## 3) Trabalhando com camadas de dados

In [ ]:
aldeias_indigenas.head()

Capturando a geometria de um dado

In [ ]:
aldeia = aldeias_indigenas.loc[11, 'geometry']

Verificando as distancias entre queimadas de risco alto e as aldeias

In [ ]:
dist_aldeia = queimadas_risco_alto.distance(aldeia)
dist_aldeia

In [ ]:
dist_aldeia.min()

Agrupando os dados -> queimadas_2020 e brasil_malhas_municipios

spacial join

In [ ]:
agregados = gpd.sjoin(queimadas_risco_alto, brasil_malhas_municipios, op='within')

In [ ]:
agregados.head()

In [ ]:
queimadas_muni = agregados.groupby(by='NM_MUN').size()
queimadas_muni = queimadas_muni.sort_values(ascending=False)
queimadas_muni

## 4) Trabalhando com o Folium

In [ ]:
coord_centro_brasil = [-14.235004, -51.925282]

In [ ]:
mapa = folium.Map(location=coord_centro_brasil, zoom_start=4)
mapa

In [ ]:
queimadas_muni

In [ ]:
queimadas_muni_df = queimadas_muni.to_frame()
queimadas_muni_df

In [ ]:
queimadas_muni_df = queimadas_muni_df.reset_index()
queimadas_muni_df.columns = ['NM_MUN', 'qtd_queimadas']
queimadas_muni_df

In [ ]:
type(queimadas_muni_df)

In [ ]:
queimadas_muni_geom_df = pd.merge(brasil_malhas_municipios, queimadas_muni_df, on='NM_MUN')
queimadas_muni_geom_df

In [ ]:
type(queimadas_muni_geom_df)

In [ ]:
queimadas_muni_geom_RR_df = queimadas_muni_geom_df[queimadas_muni_geom_df['SIGLA_UF'] == 'RR']
queimadas_muni_geom_RR_df

Mapa Cloropletico ele vai variando as cores de acordo com o acontecimento observado.

In [ ]:
folium.Choropleth(geo_data = queimadas_muni_geom_RR_df,
                  name = 'geometry',
                  data = queimadas_muni_geom_RR_df,
                  columns = ['NM_MUN', 'qtd_queimadas'],
                  key_on = 'feature.properties.NM_MUN',
                  fill_color = 'YlGn',
                  fill_opacity = 0.75,
                  line_opacity = 0.5,
                  legend_name = 'Quantidade de queimadas por município').add_to(mapa)

mapa

In [ ]:
mapa.save('meu_mapa.html')